# Álgebra Lineal

En esta clase abordaremos operaciones de álgebra lineal en Python, en particular el módulo de Álgebra lineal de la biblioteca `SciPy` ([`scipy.linalg`](https://docs.scipy.org/doc/scipy/reference/linalg.html#module-scipy.linalg)).

## Previos

En primer lugar, `scipy.linalg` contiene todas las funciones de [`numpy.linalg`](https://www.numpy.org/devdocs/reference/routines.linalg.html), en ocasiones agregando funcionalidades, pero posee algunas funciones adicionales. Además podrían existir diferencias de velocidad de cómputo dependiendo de como NumPy fue instalado, por lo que se recomiendo utilizar SciPy para tareas de álgebra lineal.

Como nuevamente no buscamos inventar la rueda de nuevo, esta clase está basada en el tutorial del módulo de álgebra lineal de Scipy ([link](https://docs.scipy.org/doc/scipy/reference/tutorial/linalg.html)).

Antes de comenzar, un tema que causa confusiones para matemáticos es el hecho que los objetos `2-D numpy.array` (de dos dimensiones) __no__ son matrices como se suelen estudiar en cursos de matemática. Un ejemplo de aquello es que la multiplicación de dos `2-D numpy.array` no tienen que coincidir para una multiplicación matricial y además la multiplicación por defecto es elemento a elemento.

En algún momento de la historia de NumPy se optó por una clase especial [`numpy.matrix`](https://numpy.org/devdocs/reference/generated/numpy.matrix.html#numpy.matrix) pero que al parecer trajo más confusiones consigo e incluso será removida en futuras versiones.

## Rutinas Básicas

### Inversa

Sabemos que la inversa de la matriz $\mathbf{A}$ es la matriz $\mathbf{A}^{-1}$, tal que

$$
    \mathbf{AA^{-1}}=\mathbf{A^{-1}A}=\mathbf{I}
$$

donde $\mathbf{I}$ es la matriz identidad.

En SciPy, la matriz inversa de un NumPy array ``A`` se obtiene utilizando `linalg.inv(A)` o ``A.I``.

In [1]:
import numpy as np
from scipy import linalg

In [2]:
A = np.array([[1,3,5],[2,5,1],[2,3,8]])
A

array([[1, 3, 5],
       [2, 5, 1],
       [2, 3, 8]])

In [3]:
linalg.inv(A)

array([[-1.48,  0.36,  0.88],
       [ 0.56,  0.08, -0.36],
       [ 0.16, -0.12,  0.04]])

In [4]:
A.dot(linalg.inv(A))  # double check

array([[ 1.00000000e+00, -1.11022302e-16,  4.85722573e-17],
       [ 3.05311332e-16,  1.00000000e+00,  7.63278329e-17],
       [ 2.22044605e-16, -1.11022302e-16,  1.00000000e+00]])

In [5]:
linalg.inv(A).dot(A)  # double double check

array([[1.00000000e+00, 3.33066907e-16, 0.00000000e+00],
       [0.00000000e+00, 1.00000000e+00, 4.44089210e-16],
       [1.38777878e-17, 6.93889390e-18, 1.00000000e+00]])

```{attention} Notar que hay un error de precisión pues está relacionado con la precisión de número flotante de NumPy.
```

### Sistema de ecuaciones

Es tan sencillo como utilizar el comando `linalg.solve` que necesita como inputs un matriz y un vector, para luego retornar un vector solución. Por ejemplo para el sistema

$$
    \begin{eqnarray*} x + 3y + 5z & = & 10 \\
                      2x + 5y + z & = & 8  \\
                      2x + 3y + 8z & = & 3
    \end{eqnarray*}
$$

se puede escribir en forma matricial

$$
\left[\begin{array}{ccc} 1 & 3 & 5\\ 2 & 5 & 1\\ 2 & 3 & 8\end{array}\right] \left[\begin{array}{c} x\\ y\\ z\end{array}\right]=\left[\begin{array}{c} 10\\ 8\\ 3\end{array}\right]
$$

por lo que es posible resolverlo utilizando la inversa

$$
\left[\begin{array}{c} x\\ y\\ z\end{array}\right]=\left[\begin{array}{ccc} 1 & 3 & 5\\ 2 & 5 & 1\\ 2 & 3 & 8\end{array}\right]^{-1}\left[\begin{array}{c} 10\\ 8\\ 3\end{array}\right]=\frac{1}{25}\left[\begin{array}{c} -232\\ 129\\ 19\end{array}\right]=\left[\begin{array}{c} -9.28\\ 5.16\\ 0.76\end{array}\right].
$$

In [6]:
A = np.array([[1, 2], [3, 4]])
A

array([[1, 2],
       [3, 4]])

In [7]:
b = np.array([[5], [6]])
b

array([[5],
       [6]])

In [8]:
linalg.inv(A).dot(b)

array([[-4. ],
       [ 4.5]])

In [9]:
A.dot(linalg.inv(A).dot(b)) - b  # check

array([[0.],
       [0.]])

Sin embargo `linalg.solve` ofrece una forma de resolver es sistema mucho más amigable e inclusive más eficiente.

In [10]:
linalg.solve(A, b)

array([[-4. ],
       [ 4.5]])

In [11]:
A.dot(linalg.solve(A, b)) - b  # check

array([[0.],
       [0.]])

### Determinante

No hay mucho que decir aquí, más que tener en consideración problemas de presición.

In [12]:
A = np.array([[1,2],[3,4]])
A

array([[1, 2],
       [3, 4]])

In [13]:
linalg.det(A)

-2.0

### Normas

SciPy cuenta con una gran variedad de normas que puedes ser calculadas a través de un argumento en `linalg.norm`. Lo interesante es que esta función puede ser utilizada para elementos 1-D (vectores) o 2-D (matrices), además de tener un argumento opcional `order` para seleccionar una norma en particular (por defecto es 2).

Sea $x$ un vector

$$
\left\Vert \mathbf{x}\right\Vert =\left\{ \begin{array}{cc} \max\left|x_{i}\right| & \textrm{ord}=\textrm{inf}\\ \min\left|x_{i}\right| & \textrm{ord}=-\textrm{inf}\\ \left(\sum_{i}\left|x_{i}\right|^{\textrm{ord}}\right)^{1/\textrm{ord}} & \left|\textrm{ord}\right|<\infty.\end{array}\right.
$$

Sea $A$ una matrix

$$
    \left\Vert \mathbf{A}\right\Vert =\left\{ \begin{array}{cc} \max_{i}\sum_{j}\left|a_{ij}\right| & \textrm{ord}=\textrm{inf}\\ \min_{i}\sum_{j}\left|a_{ij}\right| & \textrm{ord}=-\textrm{inf}\\ \max_{j}\sum_{i}\left|a_{ij}\right| & \textrm{ord}=1\\ \min_{j}\sum_{i}\left|a_{ij}\right| & \textrm{ord}=-1\\ \max\sigma_{i} & \textrm{ord}=2\\ \min\sigma_{i} & \textrm{ord}=-2\\ \sqrt{\textrm{trace}\left(\mathbf{A}^{H}\mathbf{A}\right)} & \textrm{ord}=\textrm{'fro'}\end{array}\right.
$$

donde $\sigma_{i}$ son los valores singulares de $\mathbf{A}$.

In [14]:
A=np.array([[1,2],[3,4]])
A

array([[1, 2],
       [3, 4]])

In [15]:
linalg.norm(A)

5.477225575051661

In [16]:
linalg.norm(A,'fro') # frobenius norm is the default

5.477225575051661

In [17]:
linalg.norm(A,1) # L1 norm (max column sum)

6.0

In [18]:
linalg.norm(A,-1)

4.0

In [19]:
linalg.norm(A,np.inf) # L inf norm (max row sum)

7.0

## Descomposiciones

### Espectral

La descomposición espectral es usual de encontrar en múltiples campos de la ingeniería y la matemática dadas las interpretaciones que se le pueden dar a los valores y vectores propios.

Se define la descomposición espectral para la matriz cuadrada $\mathbf{A}$ como

$$
\mathbf{Av}=\lambda\mathbf{v}
$$

donde $\lambda$ es un escalar y $\mathbf{v}$ un vector, denominados valor y vector propio respectivamente.

In [20]:
A = np.array([[1, 2], [3, 4]])
la, v = linalg.eig(A)
l1, l2 = la
print(l1, l2)   # eigenvalues

(-0.3722813232690143+0j) (5.372281323269014+0j)


In [21]:
print(v[:, 0])   # first eigenvector

[-0.82456484  0.56576746]


In [22]:
print(v[:, 1])   # second eigenvector

[-0.41597356 -0.90937671]


In [23]:
print(np.sum(abs(v**2), axis=0))  # eigenvectors are unitary

[1. 1.]


In [24]:
v1 = np.array(v[:, 0]).T
print(linalg.norm(A.dot(v1) - l1*v1))  # check the computation

5.551115123125783e-17


### Valor Singular

La descomposición valor singular (SVD) puede ser pensada como una extensión a la espectral cuando las matrices no son cuadradas. Sea $\mathbf{A}$ de tamaño $M\times N$, su descomposición valor singular es 

$$
    \mathbf{A=U}\boldsymbol{\Sigma}\mathbf{V}^{H}
$$

aquí $\mathbf{U}$ y $\mathbf{V}$ son matrices unitarias de tamaño $M \times M$ y $N \times N$ respectivamente, mientras que $\mathbf{\boldsymbol{\Sigma}}$ es una matriz $M \times N$ donde los elementos de la diagonal principal son no nulos y se denotan usualmente como valores singulares, mientras que fuera de la diagonal sus valores son cero.

__Matriz unitaria__: Sea $\mathbf{D}$ tal que $\mathbf{D}^H \mathbf{D} = \mathbf{D}\mathbf{D}^H = \mathbf{I}$, es decir $\mathbf{D}^{-1} = \mathbf{D}^H$.

In [25]:
A = np.array([[1,2,3],[4,5,6]])
A

array([[1, 2, 3],
       [4, 5, 6]])

In [26]:
M,N = A.shape
U,s,Vh = linalg.svd(A)
Sig = linalg.diagsvd(s,M,N)
U, Vh = U, Vh
U

array([[-0.3863177 , -0.92236578],
       [-0.92236578,  0.3863177 ]])

In [27]:
Sig

array([[9.508032  , 0.        , 0.        ],
       [0.        , 0.77286964, 0.        ]])

In [28]:
Vh

array([[-0.42866713, -0.56630692, -0.7039467 ],
       [ 0.80596391,  0.11238241, -0.58119908],
       [ 0.40824829, -0.81649658,  0.40824829]])

In [29]:
U.dot(Sig.dot(Vh)) #check computation

array([[1., 2., 3.],
       [4., 5., 6.]])

### LU

La descomposición LU de una matriz $\mathbf{A}$ de tamaño $M\times N$ es

$$
    \mathbf{A}=\mathbf{P}\,\mathbf{L}\,\mathbf{U},
$$

donde $\mathbf{P}$ es una matriz de permutación de filas para la matriz identidad de $M\times M$, $\mathbf{L}$ es triangular inferior $M\times K$ con $K=\min\left(M,N\right)$) con valores unitarios en la diagonal y $\mathbf{U}$ es triangular superior.

In [30]:
A = np.array(
    [
        [2, 5, 8, 7],
        [5, 2, 2, 8],
        [7, 5, 6, 6],
        [5, 4, 4, 8]]
)
p, l, u = linalg.lu(A)

In [31]:
l

array([[ 1.        ,  0.        ,  0.        ,  0.        ],
       [ 0.28571429,  1.        ,  0.        ,  0.        ],
       [ 0.71428571,  0.12      ,  1.        ,  0.        ],
       [ 0.71428571, -0.44      , -0.46153846,  1.        ]])

In [32]:
u

array([[ 7.        ,  5.        ,  6.        ,  6.        ],
       [ 0.        ,  3.57142857,  6.28571429,  5.28571429],
       [ 0.        ,  0.        , -1.04      ,  3.08      ],
       [ 0.        ,  0.        ,  0.        ,  7.46153846]])

In [33]:
np.allclose(A - p @ l @ u, np.zeros((4, 4)))

True

### Cholesky

La descomposición Cholesky es un caso especial de la descomposición LU aplicable a matrices hermitianas definidas positivas. Cuando $\mathbf{A}=\mathbf{A}^{H}$ y $\mathbf{x}^{H}\mathbf{Ax}\geq 0$ para todo $\mathbf{x}$,

$$
    \begin{eqnarray*} \mathbf{A} & = & \mathbf{U}^{H}\mathbf{U}\\ \mathbf{A} & = & \mathbf{L}\mathbf{L}^{H}\end{eqnarray*},
$$

donde $\mathbf{L}$ es triangular inferior y $\mathbf{U}$ es triangular superior. 

__Matriz hermitiana:__ Sea $\mathbf{D}$ tal que $\mathbf{D}^H = \mathbf{D}$.

In [34]:
A = np.array([[1,-2j],[2j,5]])
L = linalg.cholesky(A, lower=True)

In [35]:
L

array([[1.+0.j, 0.+0.j],
       [0.+2.j, 1.+0.j]])

In [36]:
L @ L.T.conj()

array([[1.+0.j, 0.-2.j],
       [0.+2.j, 5.+0.j]])

In [37]:
np.allclose(A, L @ L.T.conj())

True

### QR

La descomposición QR es aplicable a cualquier matriz $M\times N$, obteniendo una matriz unitaria $\mathbf{Q}$ de tamaño $M\times M$ y una matriz trapezoidal superior  $\mathbf{R}$ de $M\times N$ tal que

$$
\mathbf{A=QR}
$$


In [38]:
A = np.random.randn(9, 6)
q, r = linalg.qr(A)

In [39]:
np.allclose(A, np.dot(q, r))

True

In [40]:
q.shape, r.shape

((9, 9), (9, 6))